In [1]:
from datetime import datetime
from pathlib import Path
import pandas as pd

In [48]:
# get the start time from log file in User_data
User_data_path = Path.cwd() / 'User_data'
folder_path = User_data_path / '12345'
session_path = folder_path / 'session_1'
log_file_path = 'User_data/12345/session_1/12345_session_1_22-03-2023_21-53.log'
data_file_path = 'User_data/12345/session_1/12345_session_1_22-03-2023_21-53.txt'
date = log_file_path.split('_')[5]


<class 'pathlib.PosixPath'>


In [50]:

def get_start_end_time_logfile(log_file_path, date):
    '''
    This function gets the start time and end time from the log file.
    Input: log file path and date
    Returns: start time of the log file in datetime format
    '''
    log_start_time = ''
    #open log file
    with open(log_file_path, 'r') as file:
        #get the start time
        file.seek(0)  # Move pointer to beginning of file
        for i in range(2):  # Move pointer to end of second line
            file.readline()
        third_line_pos = file.tell()  # Get position of beginning of third line
        file.seek(third_line_pos)  # Move pointer to beginning of third line
        third_line = file.readline()  # Read third line
        log_start_time = third_line.split('\t')[0]
        log_start_time = date+" "+log_start_time

        file.seek(0)
        # Read the contents of the file
        lines = file.readlines()

        # Get the index of the second last line
        second_last_line_index = len(lines) - 2

        # Move the file pointer to the start of the second last line
        file.seek(0)
        file.seek(sum(len(line) for line in lines[:second_last_line_index]))

        # Read the contents of the second last line
        second_last_line = file.readline()
        log_end_time = second_last_line.split('\t')[0]
        log_end_time = date+" "+log_end_time
        file.close()
        #convert to datetime format
        exp_start_time = datetime.strptime(log_start_time, '%d-%m-%Y %H:%M:%S.%f')
        exp_end_time = datetime.strptime(log_end_time, '%d-%m-%Y %H:%M:%S.%f')
        return log_start_time, log_end_time

In [25]:
exp_start_time, exp_end_time = get_start_end_time_logfile(log_file_path, date)


2023-03-22 21:49:43.711470 2023-03-22 21:52:48.815724


In [54]:
# create a dataframe from the data file
def remove_timestamp(data_file_path,exp_start_time, exp_end_time):
    '''
    This function removes the data outside the experiment
    start and end time using the timestamp.
    Input: data file path, start time and end time in datetime format
    Returns: dataframe with data within the experiment time
    '''
    df = pd.read_csv(data_file_path, header=4)
    df[' Timestamp (Formatted)'] = pd.to_datetime(df[' Timestamp (Formatted)'])
    start_idx = df.index[(df[' Timestamp (Formatted)'] >= exp_start_time) & (df['Sample Index']==0.0)].tolist()[0]
    end_idx = df.index[(df[' Timestamp (Formatted)'] >= exp_end_time) & (df['Sample Index']==0.0)].tolist()[0]
    df1 = df.iloc[start_idx:end_idx]
    return df1

# df.loc[df['Timestamp(Formatted)']>]

In [55]:
User_data_path = Path.cwd() / 'User_data'

#iterate on every folder in User_data
for folder in User_data_path.iterdir():
    #iterate on every session in the folder
    for session in folder.iterdir():
        #iterate on every file in the session
        for file in session.iterdir():
            #check if the file is a log file
            if file.suffix == '.log':
                log_file_path = Path(file)
                data_file_path = file.with_suffix('.txt')
                date = str(log_file_path).split('_')[6]
                exp_start_time, exp_end_time = get_start_end_time_logfile(log_file_path, date)
                df1 = remove_timestamp(data_file_path,exp_start_time, exp_end_time)
                df1.to_csv(file.with_suffix('.csv'), index=False)


# folder_path = User_data_path / '12345'
# session_path = folder_path / 'session_1'
# log_file_path = 'User_data/12345/session_1/12345_session_1_22-03-2023_21-53.log'
# data_file_path = 'User_data/12345/session_1/12345_session_1_22-03-2023_21-53.txt'
# date = log_file_path.split('_')[5]